import Python libraries first

In [1]:
%pip install pandas
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 75.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 55.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

# ETL framework

## extract

from NOAA FTP folders for Santa Barbara Airport weather data

In [3]:
df = pd.read_csv("https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_station/USW00023190.csv.gz",
                 compression = 'gzip',
                 names = ['station','date','datatype','value','U1','U2','U3','U4'],
                 low_memory = False)

## transform

make df DataFrame object into a useable data table

In [4]:
df["date"] = pd.to_datetime(df["date"].astype(str))
df = df.loc[(df["datatype"].isin(["TMIN","TMAX"])) & (df["date"] >= "1970-01-01"), ["date", "datatype", "value"]]
df["value"] = np.round(np.array(df["value"])/10 * 1.8 + 32, 0)
df = df.pivot(index = "date", columns = "datatype", values = "value").dropna().rename_axis(None, axis = 1).reset_index(inplace = False)
df["TAVG"] = (df["TMIN"] + df["TMAX"]) / 2
df = df.rename(columns = {"date": "Date", "TMAX": "High Temperature", "TMIN": "Low Temperature", "TAVG": "Average Temperature"})

# Create aggregated DataFrame object

Select "Date" and "Average Temperature" columns

In [5]:
df[["Date","Average Temperature"]]

,Date,Average Temperature
0,1970-01-01,46.5
1,1970-01-02,46.0
2,1970-01-03,45.5
3,1970-01-04,44.5
4,1970-01-05,46.5
...,...,...
18896,2024-04-23,56.5
18897,2024-04-24,56.0
18898,2024-04-25,55.0
18899,2024-04-26,63.0


Drop NAs from selected columns

In [6]:
df[["Date","Average Temperature"]] \
  .dropna()

,Date,Average Temperature
0,1970-01-01,46.5
1,1970-01-02,46.0
2,1970-01-03,45.5
3,1970-01-04,44.5
4,1970-01-05,46.5
...,...,...
18896,2024-04-23,56.5
18897,2024-04-24,56.0
18898,2024-04-25,55.0
18899,2024-04-26,63.0


Convert date into the end of each month and year, then group by Date as Dimension in first column, calculate mean for Average Temperatures by each monthly period

In [12]:
df[["Date","Average Temperature"]] \
  .dropna() \
    .groupby(pd.Grouper(key = 'Date', axis = 0, freq = 'ME')).mean()

,Average Temperature
Date,
1970-01-31,52.838710
1970-02-28,55.410714
1970-03-31,56.467742
1970-04-30,56.666667
1970-05-31,60.580645
...,...
2023-12-31,57.741935
2024-01-31,55.677419
2024-02-29,56.017241


Reset index of new grouped DataFrame object

In [13]:
df[["Date","Average Temperature"]] \
  .dropna() \
    .groupby(pd.Grouper(key = 'Date', axis = 0, freq = 'ME')).mean() \
      .reset_index()

,Date,Average Temperature
0,1970-01-31,52.838710
1,1970-02-28,55.410714
2,1970-03-31,56.467742
3,1970-04-30,56.666667
4,1970-05-31,60.580645
...,...,...
647,2023-12-31,57.741935
648,2024-01-31,55.677419
649,2024-02-29,56.017241
650,2024-03-31,56.548387


Display most recent 120 monthly periods

In [9]:
df[["Date","Average Temperature"]] \
  .dropna() \
    .groupby(pd.Grouper(key = 'Date', axis = 0, freq = 'ME')).mean() \
      .reset_index() \
        .tail(120)

/tmp/ipykernel_5910/4200126.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(pd.Grouper(key = 'Date', axis = 0, freq = 'M')).mean() \


,Date,Average Temperature
532,2014-05-31,64.645161
533,2014-06-30,63.800000
534,2014-07-31,69.532258
535,2014-08-31,67.741935
536,2014-09-30,69.133333
...,...,...
647,2023-12-31,57.741935
648,2024-01-31,55.677419
649,2024-02-29,56.017241
650,2024-03-31,56.548387


# Set index column to field in DataFrame

Create overall average temperatures by California city

In [16]:
temps = [["Bakersfield", 78,  53, 65.5],
        ["Burbank", 77, 55, 66],
        ["Fresno", 77,  51, 64],
        ["Long Beach", 74,  55, 64.5],
        ["Los Angeles", 70,  56, 63],
        ["Sacramento", 74,  48, 61],
        ["San Diego", 71,  57, 64],
        ["San Francisco", 66,  50, 58],
        ["Santa Barbara", 70,  49, 59.5]]
df = pd.DataFrame(temps, columns = ["Station", "High Temperature", "Low Temperature", "Average Temperature"])
df.head(10)

,Station,High Temperature,Low Temperature,Average Temperature
0,Bakersfield,78,53,65.5
1,Burbank,77,55,66.0
2,Fresno,77,51,64.0
3,Long Beach,74,55,64.5
4,Los Angeles,70,56,63.0
5,Sacramento,74,48,61.0
6,San Diego,71,57,64.0
7,San Francisco,66,50,58.0
8,Santa Barbara,70,49,59.5


In [17]:
df.set_index("Station")

,High Temperature,Low Temperature,Average Temperature
Station,,,
Bakersfield,78,53,65.5
Burbank,77,55,66.0
Fresno,77,51,64.0
Long Beach,74,55,64.5
Los Angeles,70,56,63.0
Sacramento,74,48,61.0
San Diego,71,57,64.0
San Francisco,66,50,58.0
Santa Barbara,70,49,59.5
